In [4]:
import os
import requests
import random
import pickle
KEY = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'
KEY2 = 'flLQZGa0kx8opciUA6M7R9iLPwfFPj5yFKKlCiVb'

MOST_RECENT_HR_BILL = 6536 #1490
MOST_RECENT_S_BILL = 3548  #606
CONGRESS = 114

In [3]:
import re 
from lxml import html
temp = set()
page = requests.get('https://www.congress.gov/search?q={%22type%22:%22bills%22,%22source%22:%22legislation%22,%22congress%22:%22114%22,%22bill-status%22:%22passed-one%22,%22chamber%22:%22Senate%22}&pageSize=250&page=1')
tree = html.fromstring(page.content)
bills = tree.xpath('//span[@class="result-heading"]//a/text()')
temp |= set(bills)

senate_passed_one_house = set()
for bill in temp:
    senate_passed_one_house.add(int(''.join(list(filter(str.isdigit, bill)))))

    

temp = set()    
for i in range(1, 10):
    page = requests.get('https://www.congress.gov/search?pageSort=documentNumber:asc&q={%22congress%22:%22114%22,%22type%22:%22bills%22,%22bill-status%22:[%22passed-one%22],%22chamber%22:%22House%22}&pageSize=250&page=' + str(i))
    tree = html.fromstring(page.content)
    bills = tree.xpath('//span[@class="result-heading"]//a/text()')
    temp |= set(bills)
    
hr_passed_one_house = set()
for bill in temp:
    hr_passed_one_house.add(int(''.join(list(filter(str.isdigit, bill)))))


KeyboardInterrupt: 

In [23]:
import networkx as nx
%matplotlib inline


def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

def getSponsors(bill_id, congress, progress = 'introduced'):
    '''Returns (PrimarySponsor, [cosponsors]) for specified bill_id and congress strings. Returns () if faulty'''
    '''progress = 'introduced', 'became law'                                                                '''
    edge = getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/' + bill_id + '/cosponsors.json')
    toReturn = False
    if edge['status'] == 'OK':
        if progress =='introduced':
            toReturn = True
        elif progress == 'became law':
            toReturn = edge['results'][0]['latest_major_action'][:17] == 'Became Public Law'           
                
    if toReturn:
        return (edge['results'][0]['sponsor_id'], [cosponsor['cosponsor_id'] for cosponsor in edge['results'][0]['cosponsors']])
    else:
        return ()
    
def getMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'],0) for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],0) for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)

def getNameIDPairing(congress):
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'], senator['first_name'] + ' ' + senator['last_name'] + '(' + senator['party'] + ')') for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],rep['first_name'] + ' ' + rep['last_name'] + '(' + rep['party'] + ')') for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)
    
    
def buildGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph

def buildFilteredGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in sorted(list(hr_passed_one_house)):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in sorted(list(senate_passed_one_house)):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph

def reverseCurrentGraph(curr_graph):
    new_graph = {}
    for m in curr_graph:
        new_graph[m] = []
    for m in curr_graph:
        cosponsored = curr_graph[m]
        for c in cosponsored:
            new_graph[c].append(m)
    return new_graph


def ranker(members, graph, p=.15, iterations=1000000):
    assert len(graph) == len(members)
    member_list = list(graph.keys())
    current = random.choice(member_list)
    members[current] += 1
    for i in range(iterations):
#         if i % 10 == 0:
#             print("Iteration: {0}".format(i))
        if len(graph[current]) == 0 or random.random() < p:
            current = random.choice(member_list)
#             print("reset")

        else:
            neighbors = graph[current]
            current = neighbors[random.randint(0, len(neighbors)-1)]
#             print("Continue")
        members[current] += 1
    return members


def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [35]:
# members = getMembers(CONGRESS)
# graph = buildGraph(members, CONGRESS)
names = getNameIDPairing(CONGRESS)
# filteredGraph = buildFilteredGraph(members, CONGRESS)

# save_obj(filteredGraph, 'congress' + str(CONGRESS) + '_filteredGraphAtLeastOneHouse')
# save_obj(members, 'congress' + str(CONGRESS) + '_members')
# save_obj(graph, 'congress' + str(CONGRESS) + '_graph')
# save_obj(names, 'congress' + str(CONGRESS) + '_names')

In [40]:
filteredGraph = load_obj('congress' + str(CONGRESS) + '_filteredGraph')
graph = load_obj('congress' + str(CONGRESS) + '_graph')
graph = reverseCurrentGraph(graph)
members = load_obj('congress' + str(CONGRESS) + '_members')
names = load_obj('congress' + str(CONGRESS) + '_names')

In [41]:
def drawGraph(graph):
    G = nx.Graph()
    for u in graph:
        for v in graph[u]:
            G.add_edge(u, v)
#     pos = nx.spring_layout(G,k=0.15,iterations=100, scale=3)
#     nx.draw(G, pos)
    return list(nx.connected_component_subgraphs(G))


graphs = drawGraph(graph)
print(nx.number_of_nodes(graphs[0]))
print(nx.number_of_nodes(graphs[1]))
print(graphs[1].nodes)

446
100
<bound method Graph.nodes of <networkx.classes.graph.Graph object at 0x00000286FD01B470>>


In [42]:
for n in graphs[1].nodes():
    print(names[n])

David Perdue(R)
Gary Peters(D)
Al Franken(D)
Claire McCaskill(D)
Barbara Boxer(D)
John Boozman(R)
James Inhofe(R)
Martin Heinrich(D)
Pat Roberts(R)
Thad Cochran(R)
Christopher Coons(D)
Mazie Hirono(D)
Orrin Hatch(R)
Tim Scott(R)
Rand Paul(R)
Bob Casey(D)
Dianne Feinstein(D)
Dean Heller(R)
Jon Tester(D)
Bob Corker(R)
John Cornyn(R)
Michael Crapo(R)
Charles Grassley(R)
Cory Gardner(R)
Deb Fischer(R)
Richard Durbin(D)
Roger Wicker(R)
Bill Nelson(D)
Jeff Flake(R)
Michael Bennet(D)
Bill Cassidy(R)
Joe Donnelly(D)
Tammy Baldwin(D)
Elizabeth Warren(D)
John Thune(R)
Mark Warner(D)
Joe Manchin III(D)
Robert Menendez(D)
Ron Johnson(R)
Patrick Leahy(D)
Kelly Ayotte(R)
Lamar Alexander(R)
Edward Markey(D)
Marco Rubio(R)
Richard Shelby(R)
Mark Kirk(R)
Christopher Murphy(D)
Harry Reid(D)
Rob Portman(R)
John McCain(R)
Susan Collins(R)
James Lankford(R)
Jeff Merkley(D)
Tom Cotton(R)
Angus King(I)
Amy Klobuchar(D)
Steve Daines(R)
Jerry Moran(R)
Ted Cruz(R)
Joni Ernst(R)
Charles Schumer(D)
David Vitter(R

In [15]:
members = load_obj('congress' + str(CONGRESS) + '_members')
ranking = ranker(members, graph,.75, 100000)
member_ranks = [pair[0] for pair in sorted(members.items(), key = lambda item: item[1], reverse=True)]
top_score = members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,members[member_ranks[i]]/top_score, names[member_ranks[i]]))

1 1.000000: Diane Black(R)
2 0.926540: Christopher Smith(R)
3 0.862559: Erik Paulsen(R)
4 0.857820: Rosa DeLauro(D)
5 0.834123: Sam Johnson(R)
6 0.819905: Matt Cartwright(D)
7 0.808057: Charles Boustany Jr.(R)
8 0.791469: Lynn Jenkins(R)
9 0.777251: Tom Reed(R)
10 0.774882: Carolyn Maloney(D)
11 0.765403: Brett Guthrie(R)
12 0.748815: Paul Gosar(R)
13 0.736967: Katherine Clark(D)
14 0.718009: Kevin Brady(R)
15 0.713270: Peter Roskam(R)
16 0.703791: David Cicilline(D)
17 0.689573: Tom Price(R)
18 0.684834: Chris Gibson(R)
19 0.668246: Michael McCaul(R)
20 0.656398: Orrin Hatch(R)
21 0.651659: Martha McSally(R)
22 0.651659: Robert Goodlatte(R)
23 0.646919: Tim Walberg(R)
24 0.642180: Ted Poe(R)
25 0.635071: Raúl Grijalva(D)
26 0.635071: Benjamin Cardin(D)
27 0.632701: F. Sensenbrenner(R)
28 0.630332: Bill Posey(R)
29 0.627962: Ted Yoho(R)
30 0.625592: Joe Courtney(D)
31 0.625592: Dave Reichert(R)
32 0.625592: Edward Royce(R)
33 0.623223: Robert Scott(D)
34 0.620853: Trent Franks(R)
35 0.

In [68]:
test = getJSON('https://api.propublica.org/congress/v1/members/N000147/bills/introduced.json')

In [139]:
test2 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6477/cosponsors.json')

In [135]:
test3 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6431/cosponsors.json')

In [136]:
test3 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6431/cosponsors.json')
test3['results'][0]['latest_major_action'][:17] == 'Became Public Law'

True

In [157]:
test4 = load_obj('congress' + str(CONGRESS) + '_filteredGraph')

In [3]:
len(load_obj('congress114_members'))

548

In [4]:
getJSON('https://api.propublica.org/congress/v1/'+ str(114) +'/house/members.json')['results'][0]['members'][0]

# print(propub)

{'api_uri': 'https://api.propublica.org/congress/v1/members/A000055.json',
 'crp_id': '',
 'cspan_id': '45516',
 'district': '4',
 'domain': 'aderholt.house.gov',
 'dw_nominate': '',
 'facebook_account': 'RobertAderholt',
 'first_name': 'Robert',
 'google_entity_id': '/m/024p03',
 'govtrack_id': '400004',
 'icpsr_id': '29701',
 'id': 'A000055',
 'ideal_point': '',
 'in_office': 'false',
 'last_name': 'Aderholt',
 'leadership_role': '',
 'middle_name': 'B.',
 'missed_votes': '22',
 'missed_votes_pct': '1.66',
 'next_election': '2016',
 'ocd_id': 'ocd-division/country:us/state:al/cd:4',
 'office': '',
 'party': 'R',
 'phone': '',
 'rss_url': 'https://aderholt.house.gov/rss.xml',
 'seniority': '20',
 'state': 'AL',
 'total_present': '0',
 'total_votes': '1325',
 'twitter_account': 'Robert_Aderholt',
 'url': 'https://aderholt.house.gov',
 'votes_with_party_pct': '94.40',
 'votesmart_id': ''}

In [ ]:
# # print(propub)
# print(len(propub))
# propub[0]['govtrack_id']

for i in range(1, 448):
    govtrack = propub[i]['govtrack_id']
    requests.get('https://www.govtrack.us/api/v2/person/' + str(govtrack)).json()